# Building AI Agent Bot With RAG, Langchain, and Reasoning Engine From Scratch

## Setup

* This notebook will walk you through some required setup that you need to do before starting with the materials.

* It is highly recommended to use new virtual environment when running jupyter notebook for this workshop.

## Required Software Installed Locally

* Python version 3.9, 3.10, or 3.11. **Python3.12 will not work**.

* If you are using VSCode, please install Jupyter Notebook extensions.

* Jupyter notebook. Please follow this [installation guide](https://docs.jupyter.org/en/stable/install.html). You may choose whether you want to install classic jupyter notebook or jupyterlab (the next-gen web ui for jupyter)

    * [Classic jupyter notebook installation guide](https://docs.jupyter.org/en/stable/install/notebook-classic.html)

    * [Jupyterlab installation guide](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

* Google Cloud CLI. Please follow this [installation guide](https://cloud.google.com/sdk/docs/install-sdk)

### Installing dependencies

In [ ]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain
langchain_core
langchain_community
langchain-google-vertexai==2.0.8
cloudpickle
pydantic==2.9.2
langchain-google-community
google-cloud-discoveryengine
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]
langchain-google-cloud-sql-pg
numpy
pandas
pgvector
psycopg2-binary
langchain-openai
langgraph
traceloop-sdk
opentelemetry-instrumentation-google-generativeai
opentelemetry-instrumentation-langchain
opentelemetry-instrumentation-vertexai
python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!pip install --upgrade -r requirements.txt

in case you are facing issue with installing psycopg2, please run the following command (linux only):

```
sudo apt update
sudo apt install python3-dev libpq-dev
```

You will require to restart the jupyter kernel once the dependency installed.

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell. 

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [21]:
# #@markdown ###Authenticate your Google Cloud Account and enable APIs.
# # Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

## Accessing Google Cloud Credit

Please redeem your $5 USD credit that you can use for this workshop. Link for this, will be shared on the class room.

The instruction given will also require you to create a new GCP project. Create one!

## Enabling Google Service API

Before creating cloud resources (e.g. database, cloudrun services, reasoning engine, etc), first we must enable the services api.

In [ ]:
# @markdown Replace the required placeholder text below. You can modify any other default values, if you like.

# please change the project id into your gcp project id you just created. 
project_id = "imrenagi-gemini-experiment"  # @param {type:"string"}

# you can leave this the same.
region = "us-central1"  # @param {type:"string"}

!gcloud config set project {project_id} --quiet

In [ ]:
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

Here, we will enable few services:

* `aiplatform.googleapis.com` -> used for using Gemini LLM and reasoning engine
* `run.googleapis.com` -> used for deploying to cloud run
* `cloudbuild.googleapis.com` -> used for building docker image and perform the deployment

In [ ]:
!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable run.googleapis.com 
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable sqladmin.googleapis.com
!gcloud services enable cloudtrace.googleapis.com

!gcloud beta services identity create --service=aiplatform.googleapis.com --project={project_id}

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:{project_number}-compute@developer.gserviceaccount.com \
    --role="roles/cloudbuild.builds.builder"

# Deploying Dummy API server

Later on this workshop, you will be using your AI agent to interact with api in order to get detail about an online course you provide as well as to create purchase request. Hence, we will deploy the simple stupid API to cloudrun.

If you want to see the detail, you can check the `api/` directory.

Now let's deploy the Go API to cloud run:

In [ ]:
# change this registry name with an unique name
registry_name = "imrenagi-gemini-experiment-registry"  # @param {type:"string"}

!gcloud artifacts repositories create {registry_name} \
      --repository-format=docker \
      --location={region} \
      --description="devfest artifact registry" \
      --immutable-tags       

registry_url = f"{region}-docker.pkg.dev/{project_id}/{registry_name}"

We will build the docker image used by the API

In [ ]:
!gcloud builds submit api --tag {registry_url}/courses-api

We will deploy the docker image to cloud run so that we can have the api up and running

In [ ]:
!gcloud run deploy courses-api --allow-unauthenticated --region {region} --quiet --image {registry_url}/courses-api

Once it is deployed, run the command to get the url of your dummy api. Take note because we will use it later:

In [ ]:
urls = !gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'
api_url = urls[0]
print(api_url)

Testing the API

In [ ]:
!curl {api_url}/courses

# Creating Staging Bucket for AI Agent

Later, when we deploy the AI Agent, we have to provide the staging gcs bucket used to store the pickle and some other configurations of our reasoning engine. So, let's create a new empty bucket. Please change `staging_bucket_name` variable below with globally unique name.

Once the bucket created, take note the name of the bucket.

In [ ]:
# change this with globaly unique name. you may add your name to make it unique. this bucket will be used later for storing the model
staging_bucket_name = "ai-agent-demo-bucket" # @param {type:"string"}

!gcloud storage buckets create gs://{staging_bucket_name} --project={project_id} --location={region} --uniform-bucket-level-access

# Data Preparation

In this workshop, we are going to use written content from [OWASP CheatSheetSeries](https://github.com/OWASP/CheatSheetSeries) as the source document for our RAG. However, to reduce the cost, I already currated few files that we are going to use in `urls` variable. Instead of using all of them, we will just use few of them and build embedding with the currated files.

The source code below will just iterate over all files within `sources` directory and create a `course_content.jsonl` file containing the file contents.

In [ ]:
import json
import uuid
import requests
from pathlib import Path

urls = [
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/Authentication_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/Authorization_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/File_Upload_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/Forgot_Password_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/Password_Storage_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/REST_Security_Cheat_Sheet.md",
    "https://raw.githubusercontent.com/OWASP/CheatSheetSeries/refs/heads/master/cheatsheets/SQL_Injection_Prevention_Cheat_Sheet.md"
]

def generate_course_content_jsonl():
    output_file = 'course_content.jsonl'
    
    with open(output_file, 'w') as jsonl_file:

        for url in urls:
            response = requests.get(url)
            if response.status_code == 200:
                content = response.text
                filename = url.split('/')[-1]                
                title = filename.replace('_', ' ').replace('.md', '')

                slug = title.lower().replace(' ', '-')
                            
                record = {
                    'id': str(uuid.uuid4()),
                    'title': title,
                    'content': content,
                    'file_path': str(url),
                    'slug': slug
                }                
                json.dump(record, jsonl_file)
                jsonl_file.write('\n')
            else:
                print(f"Failed to download content. Status code: {response.status_code}")

        
    print(f"JSONL file '{output_file}' has been generated successfully.")

generate_course_content_jsonl()


Let's see what is inside the `course_content.jsonl` file:

In [ ]:
import pandas as pd

df = pd.read_json('course_content.jsonl', lines=True)
df.head()

# Creating Embedding and Vector Store

This notebook demonstrates the process of creating embeddings and setting up a vector store for a course content retrieval system. 

It covers the following key steps:

1. Importing necessary libraries and creating and setting up database and its configurations
1. Connecting to either a Google Cloud SQL
1. Loading course content data from markdown files
1. Creating embeddings for the course content using a Google Gemini embedding model
1. Storing the embeddings in a vector database for efficient similarity search

Setting up few constants:

In [17]:
import os
instance_name = os.environ['CLOUDSQL_INSTANCE_NAME']
database_password = os.environ['DB_PASSWORD']
database_name = os.environ['DB_NAME']
database_user = os.environ['DB_USER']

# Dont update these lines below

embeddings_table_name = "course_content_embeddings"
chat_history_table_name = "chat_histories"
gemini_embedding_model = "text-embedding-004"

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"


## Setting Up PostgreSQL in Google Cloud SQL

Here will we set the default GCP project and get information about the user using the GCP account.

In [ ]:
# Grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
print(f"{current_user}")

Before sending query to database, we will have to add required permissions for our notebook so that it can access the database:

In [ ]:
print(f"Granting Cloud SQL Client role to {current_user[0]}")
# granting cloudsql client role to the current user
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Next, we are going to create new postgresql database from Google CloudSQL and create postgresql user/role which will be used to store the embeddings later on

In [ ]:
#@markdown Create and setup a Cloud SQL PostgreSQL instance, if not done already.
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found an existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={database_password} \
    --authorized-networks=0.0.0.0/0
# Create the database, if it does not exist.
out = !gcloud sql databases list --instance={instance_name} --filter="NAME:{database_name}" --format="value(NAME)"
if ''.join(out) == database_name:
  print("Database %s already exists, skipping creation." % database_name)
else:
  !gcloud sql databases create {database_name} --instance={instance_name}
# Create the database user for accessing the database.
!gcloud sql users create {database_user} \
  --instance={instance_name} \
  --password={database_password}

Here we are going to get the ip of postgresql we just created. Take note to the database host ip address.

In [ ]:
# get the ip address of the instance
ip_addresses = !gcloud sql instances describe {instance_name} --project {project_id} --format 'value(ipAddresses.ipAddress)'
# Split the IP addresses and take the first one
database_host = ip_addresses[0].split(';')[0].strip()
print(f"Using database host: {database_host}")

## Prepare the embeddings

Now, we will build the embeddings from the content we have selected. 

Before creating the embedding, we need to split the content of each files into chunks. This is most of the time required, especially when the content is toolong, because embedding has the limit for the number of input token it can accept.

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

text_splitter = MarkdownTextSplitter(
  chunk_size=1000, 
  chunk_overlap=200)

from langchain_core.documents import Document

chunked = []
for index, row in df.iterrows():
    course_content_id = row["id"]
    title = row["title"]
    content = row["content"]
    splits = text_splitter.create_documents([content])
    for s in splits:
        metadata = {"course_content_id": course_content_id, "title": title}
        doc = Document(page_content=s.page_content, metadata=metadata)
        chunked.append(doc)

chunked[0]

In [ ]:
len(chunked)

Once we have the file content chunked into smaller sizes, we are going to create embedding for each chunked and store it to cloudsql.

Now let's initialize vertex ai sdk and create the embedding services.

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

print(project_id)
# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

Now, let's construct the embeddings and store it to the database.

On the function below we are doing these steps:
1. We are initiating a PostgresEngine. This instance of PostgresEngine will be used to handle database connection as well as authentication.
1. Then, `ainit_vectorstore_table()` will create a table which will be used to store the chucked content, its embedding, and metadata.
1. We initialize the PostgresVectorStore and provide the engine as well as the embedding service.
1. For each chunked document, we call function `aadd_documents` to create embedding and create new record on the given table.

In [ ]:
!pip install pip-system-certs
!pip install --upgrade certifi urllib3

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
import uuid

async def create_vectorstore():
    engine = await PostgresEngine.afrom_instance(
        project_id,
        region,
        instance_name,
        database_name,
        user=database_user,
        password=database_password,
    )

    await engine.ainit_chat_history_table(
        table_name=chat_history_table_name
    )

    await engine.ainit_vectorstore_table(
        table_name=embeddings_table_name, vector_size=768, overwrite_existing=True
    )

    vector_store = await PostgresVectorStore.create(
        engine,
        table_name=embeddings_table_name,
        embedding_service=embeddings_service,
    )

    ids = [str(uuid.uuid4()) for i in range(len(chunked))]
    await vector_store.aadd_documents(chunked, ids=ids)

await create_vectorstore()

Once you have the vector store, you can check the content from google cloud sql data viewer.

# Retriever

Once we have data stored in cloudsql, we need to find a way to query the data. This notebook covers how we can create and use the postgresql retriever to perform similarity search.

Similar to previous section, we will try to create PostgresEngine to connect to CloudSQL instance:

In [17]:
from langchain_google_cloud_sql_pg import PostgresEngine

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user=database_password,
    password=database_password,
)

We create the vector store object by using the engine and embedding service we created earlier:

In [18]:
from langchain_google_cloud_sql_pg import PostgresVectorStore

vector_store = PostgresVectorStore.create_sync(
            pg_engine,
            table_name=embeddings_table_name,
            embedding_service=embeddings_service,
        )
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Let's try with some query:

In [ ]:
retriever.invoke("how to design forgot password?")

In [ ]:
retriever.invoke("how to design security for authentication?")